In [1]:
import wlogs as wl
import pandas as pd
from datetime import datetime

Access token retrieved.


In [2]:
report_data_container = wl.ReportDataContainer()

In [3]:
def convert_time(epoch_time_ms):
    return datetime.fromtimestamp(float(epoch_time_ms)/1000)


def convert_time_str(epoch_time_ms, fmt = "%d/%m/%Y %H:%M:%S"):
    return datetime.fromtimestamp(float(epoch_time_ms)/1000).strftime(fmt)

In [4]:
def compose_report_data_df(report_data: dict):
    report_code = report_data['code']

    actors = pd.DataFrame(report_data['masterData']['actors']).set_index(keys='id').rename(columns={'name': 'player_name'})
    actors = actors[actors['subType']!='Unknown']
    actors['report_code'] = report_code

    fights = pd.DataFrame(report_data['fights']).set_index(keys='id').rename(columns={'name': 'boss_name'})
    fights = fights[fights['size'].notna()]
    fights['report_code'] = report_code

    report_data_df = fights.merge(actors, on='report_code')

    add_metadata = {'report_date': convert_time_str(report_data['startTime'], fmt = "%d/%m/%Y"),
                    'report_start': convert_time(report_data['startTime']),
                    'report_end': convert_time(report_data['endTime'])}
    for key, value in add_metadata.items():
        report_data_df[key] = value

    return report_data_df

In [5]:
reports_data = report_data_container.extract()

reports_df = []
for report_data in reports_data.values():
    report_data_df = compose_report_data_df(report_data)
    reports_df.append(report_data_df)

progress_data = pd.concat(reports_df)

In [6]:
ulduar_boss_hm = ['Flame Leviathan', 'XT-002 Deconstructor', 'The Assembly of Iron', 'Hodir', 'Thorim', 'Freya', 'Mimiron', 'General Vezax', 'Yogg-Saron', 'Algalon the Observer']

raiders = ['Åkodah', 'Bobsatan', 'Cheekygirl', 'Drep', 'Frallak', 'Gogø', 'Krohl', 'Madess', 'Melvira', 'Memø', 'Padreleon', 'Pahâthata', 'Selbie', 'Selemental', 'Sylhastrasz', 'Tonsurédeouf']

In [7]:
progress_ulduar_10HM = progress_data[(progress_data['kill'] == True) &
                                     (progress_data['size'] == 10) &
                                     ((progress_data['boss_name']=='Algalon the Observer') | ((~progress_data['hardModeLevel'].isna()) & ((progress_data['hardModeLevel'] >= 3) | ((progress_data['report_code']=='L1hbC8dX2c7wDKx6') & (progress_data['boss_name']=='Hodir'))))) & (progress_data['boss_name'].isin(ulduar_boss_hm)) & (progress_data['player_name'].isin(raiders))].groupby(by=['boss_name', 'player_name'])['report_start'].min().reset_index()

In [8]:
boss_killed = progress_ulduar_10HM['boss_name'].unique().tolist()
boss_killed_ordered = [boss for boss in ulduar_boss_hm if boss in boss_killed]

progress_ulduar_10HM['report_start'] = progress_ulduar_10HM['report_start'].dt.strftime("%d/%m/%Y")
first_kill_ulduar_10HM = progress_ulduar_10HM.pivot(index='player_name', columns='boss_name', values='report_start')[boss_killed_ordered]

In [9]:
first_kill_ulduar_10HM.to_csv(wl.TEMP_DIRECTORY / "first_kill_ulduar_10HM.csv")

In [10]:
"Logs manquants : xa1vWmtHRZBqycgQ"

"""
{"id": 26, "name": "XT-002 Deconstructor", "difficulty": 3, "encounterID": 747, "size": 10, "hardModeLevel": 4, "startTime": 4369811, "endTime": 4541502, "kill": true, "bossPercentage": 0.0100, "lastPhase": 0, "averageItemLevel": 221.89999389648438}, {"id": 27, "name": "The Assembly of Iron", "difficulty": 3, "encounterID": 748, "size": 10, "hardModeLevel": 4, "startTime": 4369811, "endTime": 4541502, "kill": true, "bossPercentage": 0.0000, "lastPhase": 3, "averageItemLevel": 221.89999389648438}
"""

'\n{"id": 26, "name": "XT-002 Deconstructor", "difficulty": 3, "encounterID": 747, "size": 10, "hardModeLevel": 4, "startTime": 4369811, "endTime": 4541502, "kill": true, "bossPercentage": 0.0100, "lastPhase": 0, "averageItemLevel": 221.89999389648438}, {"id": 27, "name": "The Assembly of Iron", "difficulty": 3, "encounterID": 748, "size": 10, "hardModeLevel": 4, "startTime": 4369811, "endTime": 4541502, "kill": true, "bossPercentage": 0.0000, "lastPhase": 3, "averageItemLevel": 221.89999389648438}\n'

In [11]:
"Logs manquants : xa1vWmtHRZBqycgQ"

"""
{"id": 26, "name": "XT-002 Deconstructor", "difficulty": 3, "encounterID": 747, "size": 10, "hardModeLevel": 4, "startTime": 4369811, "endTime": 4541502, "kill": true, "bossPercentage": 0.0100, "lastPhase": 0, "averageItemLevel": 221.89999389648438}, {"id": 27, "name": "The Assembly of Iron", "difficulty": 3, "encounterID": 748, "size": 10, "hardModeLevel": 4, "startTime": 4369811, "endTime": 4541502, "kill": true, "bossPercentage": 0.0000, "lastPhase": 3, "averageItemLevel": 221.89999389648438}
"""

'\n{"id": 26, "name": "XT-002 Deconstructor", "difficulty": 3, "encounterID": 747, "size": 10, "hardModeLevel": 4, "startTime": 4369811, "endTime": 4541502, "kill": true, "bossPercentage": 0.0100, "lastPhase": 0, "averageItemLevel": 221.89999389648438}, {"id": 27, "name": "The Assembly of Iron", "difficulty": 3, "encounterID": 748, "size": 10, "hardModeLevel": 4, "startTime": 4369811, "endTime": 4541502, "kill": true, "bossPercentage": 0.0000, "lastPhase": 3, "averageItemLevel": 221.89999389648438}\n'